### WALMART STORE PREDICTION - FORECASTING

 The objective of this project is to predicting Walmart store sales using historical markdown data.

#1. Importing Libraries

In [31]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns
import pandas_profiling 
import seaborn as sns


from datetime import datetime

import statsmodels.api as sm
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error, mean_squared_error

C:\Users\sony\Anaconda2\lib\site-packages\pandas_profiling\plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\sony\Anaconda2\lib\runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "C:\Users\sony\Anaconda2\lib\runpy.py", line 72, in _run_code
    exec code in run_globals
  File "C:\Users\sony\Anaconda2\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sony\Anaconda2\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\sony\Anaconda2\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:

#2. Importing Dataframes

In [35]:
walmart_train = pd.read_csv('trainMerged.csv')
walmart_test = pd.read_csv('testMerged.csv')

In [36]:
print walmart_train.head()
print walmart_test.head()

   Store        Date  IsHoliday  Dept  Weekly_Sales Type    Size  Temperature  \
0     10  2010-02-05      False     1      40212.84    B  126512        54.34   
1     10  2010-02-05      False    98         74.00    B  126512        54.34   
2     10  2010-02-05      False    32      26116.73    B  126512        54.34   
3     10  2010-02-05      False    25      20779.50    B  126512        54.34   
4     10  2010-02-05      False    34      50585.42    B  126512        54.34   

   Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5  \
0       2.962        NaN        NaN        NaN        NaN        NaN   
1       2.962        NaN        NaN        NaN        NaN        NaN   
2       2.962        NaN        NaN        NaN        NaN        NaN   
3       2.962        NaN        NaN        NaN        NaN        NaN   
4       2.962        NaN        NaN        NaN        NaN        NaN   

          CPI  Unemployment  
0  126.442065         9.765  
1  126.442065       

#3. Data Analysis, manipulation and cleaning 

In [37]:
walmart_train.info()
print '--------------------------------------------------------------------------'
walmart_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
Store           421570 non-null int64
Date            421570 non-null object
IsHoliday       421570 non-null bool
Dept            421570 non-null int64
Weekly_Sales    421570 non-null float64
Type            421570 non-null object
Size            421570 non-null int64
Temperature     421570 non-null float64
Fuel_Price      421570 non-null float64
MarkDown1       150681 non-null float64
MarkDown2       111248 non-null float64
MarkDown3       137091 non-null float64
MarkDown4       134967 non-null float64
MarkDown5       151432 non-null float64
CPI             421570 non-null float64
Unemployment    421570 non-null float64
dtypes: bool(1), float64(10), int64(3), object(2)
memory usage: 48.6+ MB
--------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 15 columns):
S

In [38]:
# Mapping type varible with Integer values

In [39]:
#Train
walmart_train['Type'] = walmart_train['Type'].map({'A': 1, 'B': 2, 'C': 3})
#Test
walmart_test['Type'] = walmart_test['Type'].map({'A': 1, 'B': 2, 'C': 3})

In [41]:
# Mapping IsHoliday varible with Integer values

In [42]:
#Train
walmart_train['IsHoliday'] = walmart_train['IsHoliday'].map({True: 1, False: 0})
#Test
walmart_test['IsHoliday'] = walmart_test['IsHoliday'].map({True: 1, False: 0})

#4. Data Desciption

In [44]:
numeric_var_train=[key for key in dict(walmart_train.dtypes) if dict(walmart_train.dtypes)[key] in ['float64', 'int64', 'float32', 'int32']]
cat_var_train=[key for key in dict(walmart_train.dtypes) if dict(walmart_train.dtypes)[key] in ['object']]
# Train Numerical Data
train_num=walmart_train[numeric_var_train]

# Train Categorical Data
train_cat=walmart_train[cat_var_train]
print numeric_var_train
print cat_var_train

['CPI', 'Temperature', 'Unemployment', 'Weekly_Sales', 'MarkDown4', 'IsHoliday', 'Dept', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'Type', 'MarkDown5', 'Store', 'Size']
['Date']


In [45]:
# Creating Data audit Report
# Use a general function that returns multiple values    
def var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  x.std(), x.var(), x.min(), x.dropna().quantile(0.01), x.dropna().quantile(0.05),x.dropna().quantile(0.10),x.dropna().quantile(0.25),x.dropna().quantile(0.50),x.dropna().quantile(0.75), x.dropna().quantile(0.90),x.dropna().quantile(0.95), x.dropna().quantile(0.99),x.max()], 
                  index=['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1' , 'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])
 
num_summary=train_num.apply(lambda x: var_summary(x)).T
num_summary

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
CPI,421570.0,0.0,7.217360e+07,171.201947,182.31878,39.159276,1.533449e+03,126.064,126.111903,126.496258,128.823806,132.022667,182.31878,212.416993,219.444244,221.941558,225.473509,227.232807
Temperature,421570.0,0.0,2.533217e+07,60.090059,62.09000,18.447931,3.403262e+02,-2.060,18.300000,27.310000,33.980000,46.680000,62.09000,74.280000,83.580000,87.270000,92.810000,100.140000
Unemployment,421570.0,0.0,3.355819e+06,7.960289,7.86600,1.863296,3.471872e+00,3.879,4.156000,5.326000,5.965000,6.891000,7.86600,8.572000,9.816000,12.187000,14.180000,14.313000
Weekly_Sales,421570.0,0.0,6.737219e+09,15981.258123,7612.03000,22711.183519,5.157979e+08,-4988.940,5.000000,59.974500,291.097000,2079.650000,7612.03000,20205.852500,42845.673000,61201.951000,106479.586000,693099.360000
MarkDown4,134967.0,286603.0,4.566161e+08,3383.168256,1481.31000,6292.384031,3.959410e+07,0.220,3.970000,28.760000,108.710000,504.220000,1481.31000,3595.040000,7871.420000,12645.960000,35785.260000,67474.850000
IsHoliday,421570.0,0.0,2.966100e+04,0.070358,0.00000,0.255750,6.540827e-02,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000,1.000000,1.000000
Dept,421570.0,0.0,1.865882e+07,44.260317,37.00000,30.492054,9.297654e+02,1.000,1.000000,4.000000,7.000000,18.000000,37.00000,74.000000,92.000000,95.000000,98.000000,99.000000
Fuel_Price,421570.0,0.0,1.416908e+06,3.361027,3.45200,0.458515,2.102356e-01,2.472,2.565000,2.653000,2.720000,2.933000,3.45200,3.738000,3.917000,4.029000,4.202000,4.468000
MarkDown1,150681.0,270889.0,1.091898e+09,7246.420196,5347.45000,8291.221345,6.874435e+07,0.270,17.760000,149.190000,375.200000,2240.270000,5347.45000,9210.900000,15282.470000,21801.350000,41524.030000,88646.760000
MarkDown2,111248.0,310322.0,3.709708e+08,3334.628621,192.00000,9475.357325,8.978240e+07,-265.760,-0.990000,1.950000,6.980000,41.600000,192.00000,1926.940000,8549.740000,16497.470000,50366.600000,104519.540000


In [46]:
pandas_profiling.ProfileReport(walmart_train)

Number of variables,16
Number of observations,421570
Total Missing (%),0.0%
Total size in memory,51.5 MiB
Average record size in memory,128.0 B
Numeric,14
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [47]:
pandas_profiling.ProfileReport(walmart_test)

Number of variables,15
Number of observations,115064
Total Missing (%),0.0%
Total size in memory,13.2 MiB
Average record size in memory,120.0 B
Numeric,13
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,0
